In [ ]:
import sys
from pathlib import Path

# some preparation to make use cpnet_new in the same way we could use cpnet
project_root = Path.cwd().parents[0]
pkg_parent    = project_root / "core-periphery-detection-master"
sys.path.insert(0, str(pkg_parent))             

import cpnet_new
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
## social networks
# https://networkrepository.com/fb-pages-food.php
# https://networkrepository.com/rt-retweet.php

## biological networks
# https://networkrepository.com/bio-celegans.php
# https://networkrepository.com/bio-CE-GT.php 

## techonological
# https://networkrepository.com/inf-USAir97.php explanation https://nps.edu/web/faculty-rgera/networkprofilesummaryresource
# https://networks.skewed.de/net/webkb webkb_wisconsin_link1 

from pathlib import Path
from typing import Dict
import networkx as nx
from scipy.io import mmread

G_fb_food = nx.read_edgelist('../empirical_networks/fb-pages-food/fb-pages-food.edges', delimiter=',')
G_fb_food.remove_edges_from(nx.selfloop_edges(G_fb_food))

matrix_bio_celegans = mmread('../empirical_networks/bio-celegans/bio-celegans.mtx')
G_bio_celegans :nx.Graph= nx.from_scipy_sparse_array(matrix_bio_celegans)
G_bio_celegans.remove_edges_from(nx.selfloop_edges(G_bio_celegans))

G_bio_CE_GT :nx.Graph= nx.read_edgelist('../empirical_networks/bio-CE-GT/bio-CE-GT.edges', data=False)
G_bio_CE_GT.remove_edges_from(nx.selfloop_edges(G_bio_CE_GT))

matrix_rt_retweet = mmread('../empirical_networks/rt-retweet/rt-retweet.mtx')
G_rt_retweet :nx.Graph = nx.from_scipy_sparse_array(matrix_rt_retweet)
G_rt_retweet.remove_edges_from(nx.selfloop_edges(G_rt_retweet))

matrix_USAir = mmread('../empirical_networks/inf-USAir97/inf-USAir97.mtx')
G_USAir : nx.Graph = nx.from_scipy_sparse_array(matrix_USAir)
G_USAir.remove_edges_from(nx.selfloop_edges(G_USAir))

G_web_wisconsin :nx.Graph = nx.read_edgelist('../empirical_networks/webkb_wisconsin_link1.csv/edges.csv', delimiter=',', data=False)
G_web_wisconsin.remove_edges_from(nx.selfloop_edges(G_web_wisconsin))


graphs = {  
    'fb-pages-food':G_fb_food,
    'celegans':G_bio_celegans,
    'bio-CE-GT':G_bio_CE_GT,
    'rt-retweet':G_rt_retweet,
    'inf-USAir97':G_USAir,
    'webkb_wisconsin_link1':G_web_wisconsin
}

for fname, G in graphs.items():
    print(f"name={fname}: nodes={G.number_of_nodes()}, edges={G.number_of_edges()}")

results = {}

## detection and statistical test
for fname, G in graphs.items():

    results[fname] = {}

    alg_discrete = cpnet_new.BE()

    alg_discrete.detect(G) 

    c = alg_discrete.get_pair_id() 
    
    x = alg_discrete.get_coreness()

    sig_c, sig_x, significant, p_values = cpnet_new.qstest(
        c, x, G, alg_discrete, significance_level=0.01, num_of_thread=8
    )

    results[fname]["Core_periphery"] = c
    results[fname]["sig_c"]=sig_c 
    results[fname]["sig_x"]=sig_x 
    results[fname]["coreness"]=x
    results[fname]["significant"] = significant 
    results[fname]["p_values"] = p_values
    print(fname)
    print(len([n for n in G.nodes() if x[n]==np.int64(1)]))
    print(f"numerical analysis of network {fname} has completed, p_values are {p_values}")


bio-CE-GT.edges: nodes=924, edges=3239, name=bio-CE-GT.edges
bio-celegans.mtx: nodes=453, edges=2025, name=bio-celegans.mtx
fb-pages-food.edges: nodes=620, edges=2091, name=fb-pages-food.edges
inf-USAir97.mtx: nodes=332, edges=2126, name=inf-USAir97.mtx
rt-retweet.mtx: nodes=96, edges=117, name=rt-retweet.mtx
edges.csv: nodes=297, edges=1130, name=edges.csv


  8%|▊         | 8/100 [06:12<1:11:20, 46.53s/it]


KeyboardInterrupt: 

In [ ]:
## create plots and figures
import matplotlib.pyplot as plt
import networkx as nx

# change this variable to create the plot for a particular network
fname = "fb-pages-food"

G = graphs[fname]
c = results[fname]["Core_periphery"]
x = results[fname]["coreness"]

print(c)
print(x)
print(G.nodes())

pos = nx.spring_layout(G, seed=42)

node_colors = ["red" if x[n] == np.int64(1) else "blue" for n in G.nodes()]

core_nodes= [n for n in G.nodes() if x[n]==np.int64(1)]

core_ratio = len(core_nodes)/len(G.nodes())

print(core_ratio)
print(len(core_nodes))

plt.figure(figsize=(8, 6))

nx.draw_networkx_edges(G, pos, alpha=0.3, width=0.5)

nx.draw_networkx_nodes(
	G,
	pos,
	node_color=node_colors,
	node_size=5,
	alpha=0.8,
)

plt.axis("off")
plt.tight_layout()
plt.savefig(f"core periphery structure for {fname}", dpi=1000)
plt.close()

